##### Importation des librairies nécessaires pour le script

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import requests
import json
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np
import datetime as dt  #pour l'ajout de la date de l'extraction de données

In [2]:
#pd.__version__

Scraping de la liste des films français / langue française sorties entre 2019 et 2021 à l'aide d'une fonction

In [3]:
url_mdb19 = 'https://www.imdb.com/search/title/?title_type=feature&release_date=2019-01-01,2019-12-31&countries=fr&languages=fr&view=simple&start='
url_mdb20 = 'https://www.imdb.com/search/title/?title_type=feature&release_date=2020-01-01,2020-12-31&countries=fr&languages=fr&view=simple&start='
url_mdb21 = 'https://www.imdb.com/search/title/?title_type=feature&release_date=2021-01-01,2021-12-31&countries=fr&languages=fr&view=simple&start='

def liste_films(url):
    liste_lib = BeautifulSoup(requests.get(url).content,'html.parser')
    #print(liste_lib.prettify())
    #for img in lib_script=liste_lib.find_all("img", {"class":"loadlate"}):
        

    for link in liste_lib.find_all("span", {"class":"lister-item-header"}):
        dico_inter=[]
        for l in link.find('a'):
            #print(l)
            dico_inter.append(l)
        fiche="https://www.imdb.com"+str(link.a.get('href'))+"?ref_=adv_li_tt"
        dico_inter.append(fiche)
        dico_inter.append(link.find("span",{"class":"lister-item-year"}).string)
        dico.append(dico_inter)    
        

Lancer la fonction à l'aide d'une boucle pour s'assurer de scrapper toutes les pages réponses

In [4]:
dico=[] #liste des films depuis IMDB
liste_url=[url_mdb19,url_mdb20,url_mdb21]
liste_page=['1','51','101','151','201','251','301'] #pour m'assurer que je scrappe bien toutes les pasges de résultats
for url in liste_url :
    for lp in liste_page:
        url_concat=url+lp+'&ref_=adv_nxt'
        #print(url_concat)
        liste_films(url_concat)

#dico

In [5]:
###POUR TEST HORS FONCTION
#dico=[]
#liste_page=['1','51','101','151','201','251','301']
#
#liste_lib = BeautifulSoup(requests.get(url_mdb21).content,'html.parser')
#
#for link in liste_lib.find_all("span", {"class":"lister-item-header"}):
#    dico_inter=[]
#    for l in link.find('a'):
#        #print(l)
#        dico_inter.append(l)
#    fiche="https://www.imdb.com"+str(link.a.get('href'))+"?ref_=adv_li_tt"
#    dico_inter.append(fiche)
#    dico_inter.append(link.find("span",{"class":"lister-item-year"}).string)
#    dico.append(dico_inter)    
#        
#        #print(l.get('href'),l.get('title'))
#dico
#

In [19]:
#print(liste_lib.prettify())
len(dico)

875

Création du dataframe où je nettoie la colonne "annee_sortie" et où j'isole l'id de l'url_fiche

In [38]:
df=pd.DataFrame(dico)
df.columns=['titre', 'url_fiche','annee_sortie']

df_toreplace_list=['(I) ','(II) ']
df_annee_toreplace_list=['(2019)','(2020)','(2021)']
for atr in df_toreplace_list:
    df['annee_sortie']=df[['annee_sortie']].apply(lambda x : x.str.replace(atr, '', regex=False))
for texte in df_annee_toreplace_list:
    df['annee_sortie']=df[['annee_sortie']].apply(lambda x : x.str.replace(texte[0], '', regex=False))
    df['annee_sortie']=df[['annee_sortie']].apply(lambda x : x.str.replace(texte[-1], '', regex=False))

df['id_imdb']=df[['url_fiche']].apply(lambda x : x.str.replace('https://www.imdb.com/title/', '', regex=True))
df['id_imdb']=df[['id_imdb']].apply(lambda x : x.str.replace('/.+', '', regex=True))
df=df.drop_duplicates()

In [39]:
df.head() #len(df)

,titre,url_fiche,annee_sortie,id_imdb
0,La belle époque,https://www.imdb.com/title/tt9172422/?ref_=adv...,2019,tt9172422
1,Anna,https://www.imdb.com/title/tt7456310/?ref_=adv...,2019,tt7456310
2,Portrait de la jeune fille en feu,https://www.imdb.com/title/tt8613070/?ref_=adv...,2019,tt8613070
3,Curiosa,https://www.imdb.com/title/tt7598076/?ref_=adv...,2019,tt7598076
4,Le Chant du loup,https://www.imdb.com/title/tt7458762/?ref_=adv...,2019,tt7458762


Je créé 2 listes : la liste des id et celle de titre

In [40]:
id_imdb=[id for id in df['id_imdb']]

In [41]:
titre_imdb=[{id:an} for id,an in zip(df['titre'],df['annee_sortie'])]

Il n'est pas possible de trouver in ID de film de movieDB (mDB) à partir de l'ID IMDB. Seul l'inverse est possible. Par contre, je peux trouver la liste des films dans mDB à l'aide des titres. Je pourrais alors avoir la liste des id.<br>
L'étape suivante sera de créer la table de correspondance Id mDB / Id IMDB pour vérifier que j'ai bien tous les films de la liste de IMBD présent dans mDB.

In [42]:
#corresp_film=[] #pour trouver les films IMDB dans la base mDB
#for x in titre_imdb:
#    for t,a in x.items():
#        url_mdb = 'http://api.themoviedb.org/3/search/movie?api_key={token}&query='+t+'&primary_release_year='+a+'&language=fr-FR&page=1&include_adult=false'
#        response_l = requests.get(url_mdb)
#        json_data_l = response_l.json()
#        corresp_film.append(json_data_l)
#        

In [43]:
#GOOD CODE A REPRENDRE - 8 MARS 2022

#corresp_film=[] #pour lister les films IMDB trouvés dans la base mDB
#flat=[] #liste titres non trouvés sur MovieDB par le biais de l'api
#for t,a,i in zip(df['titre'],df['annee_sortie'],df['id_imdb']) :
#    url_mdb = 'http://api.themoviedb.org/3/search/movie?api_key={token}&query='+t+'&primary_release_year='+a+'&language=fr-FR&page=1&include_adult=false'
#    response_l = requests.get(url_mdb)
#    json_data_l = response_l.json()
#    df_n=pd.json_normalize(json_data_l)
#    #print(df_n.columns)
#    if len(df_n.columns) == 1 :#not isinstance(df_n, pd.DataFrame)
#        flat.append({t,a,i})
#        #url_mdb = 'http://api.themoviedb.org/3/search/movie?api_key={token}&query='+t+'&primary_release_year='+a+1+'&language=fr-FR&page=1&include_adult=false'
#    else:
#        for x in df_n.results :
#            results=pd.json_normalize(x)
#            #print(type(results))
#            #print(results.id)
#            if x ==[] or isinstance(x, float) :
#                flat.append({t,a})
#            else:
#                corresp_film.append(results.id)
#            

In [44]:
corresp_film=[] #pour lister les films IMDB trouvés dans la base mDB
flat=[] #liste titres non trouvés sur MovieDB par le biais de l'api
for t,a,i in zip(df['titre'],df['annee_sortie'],df['id_imdb']) :
    url_mdb = 'http://api.themoviedb.org/3/search/movie?api_key=&query='+t+'&primary_release_year='+a+'&language=fr-FR&page=1&include_adult=false'
    response_l = requests.get(url_mdb)
    json_data_l = response_l.json()
    df_n=pd.json_normalize(json_data_l)
    #print(df_n.columns)
    if len(df_n.columns) == 1 :#not isinstance(df_n, pd.DataFrame)
        flat.append({t,a,i})   
    
    else:
        for x in df_n.results :
            results=pd.json_normalize(x)
            #print(type(results))
            #print(results.id)
            if x ==[] or isinstance(x, float) :
                flat.append({t,a,i})
            else:
                corresp_film.append(results.id)

Nb de titres non trouvés

In [45]:
len(flat)

281

In [46]:
liste_id_moviedb_temp=pd.DataFrame(corresp_film)
liste_id_moviedb_temp

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
id,595975.0,640586.0,610933.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id,484641.0,613607.0,638119.0,742965.0,653825.0,670640.0,633410.0,737028.0,790815.0,729038.0,597899.0,672389.0,780488.0,616985.0,578829.0,613851.0,788464.0,521029.0,636348.0,596258.0
id,531428.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id,487616.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id,484468.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id,893987.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id,879764.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id,889484.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id,853152.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
liste_id_moviedb=[]
for col in liste_id_moviedb_temp:
    for y in liste_id_moviedb_temp[col]:
        if len(str(y))>3:
            liste_id_moviedb.append(y)

Création de la table de correspondance Id mDB / Id IMDB

In [48]:
idimbd_moviebd_liste=[]
ap=('')## token à ajouter

for id in liste_id_moviedb:
    url_mdb = 'https://api.themoviedb.org/3/movie/'+str(id)+'/external_ids?api_key='+ap+''
    response_l = requests.get(url_mdb)
    json_data_l = response_l.json()
    #print(json_data_l)
    idimbd_moviebd_liste.append(json_data_l)

In [49]:
idimbd_moviebd_df=pd.json_normalize(idimbd_moviebd_liste)
idimbd_moviebd_df.head()

,id,imdb_id,facebook_id,instagram_id,twitter_id
0,595975,tt9172422,None,None,None
1,484641,tt7456310,annamovie,annamovie,annamovie
2,531428,tt8613070,pdljfef,portraitdelajeunefilleenfeu,Portrait_Movie
3,487616,tt7598076,None,None,None
4,484468,tt7458762,None,None,None


In [51]:
len(idimbd_moviebd_df)

1511

J'ajoute le l'id de movie Db dans le dataframe de départ;

In [52]:
idimbd_moviebd_df2=idimbd_moviebd_df.copy()
idimbd_moviebd_df2=idimbd_moviebd_df2.drop_duplicates(subset=['id','imdb_id'])
id_moviedb=[]
aa={}
for id, idc in zip(idimbd_moviebd_df2.imdb_id,idimbd_moviebd_df2.id):
    aa[id]=idc

for i in df.id_imdb:
    if i in [x for x in idimbd_moviebd_df2['imdb_id']]:
        id_moviedb.append(aa[i])
    else:
        id_moviedb.append('-')

df['id_moviedb']=id_moviedb


df

,titre,url_fiche,annee_sortie,id_imdb,id_moviedb
0,La belle époque,https://www.imdb.com/title/tt9172422/?ref_=adv...,2019,tt9172422,595975
1,Anna,https://www.imdb.com/title/tt7456310/?ref_=adv...,2019,tt7456310,484641
2,Portrait de la jeune fille en feu,https://www.imdb.com/title/tt8613070/?ref_=adv...,2019,tt8613070,531428
3,Curiosa,https://www.imdb.com/title/tt7598076/?ref_=adv...,2019,tt7598076,487616
4,Le Chant du loup,https://www.imdb.com/title/tt7458762/?ref_=adv...,2019,tt7458762,484468
...,...,...,...,...,...
870,Dark West - Director's Cut,https://www.imdb.com/title/tt15731894/?ref_=ad...,2021,tt15731894,-
871,Recuerda - un moment por ella,https://www.imdb.com/title/tt15227746/?ref_=ad...,2021,tt15227746,-
872,Klon: Nouveau Genre,https://www.imdb.com/title/tt14852738/?ref_=ad...,2021,tt14852738,-
873,Les Etoiles de Fontevraud,https://www.imdb.com/title/tt18556416/?ref_=ad...,2021,tt18556416,-


In [53]:
df2 = df.loc[df['id_moviedb']=='-']
df2

,titre,url_fiche,annee_sortie,id_imdb,id_moviedb
6,Lux Æterna,https://www.imdb.com/title/tt10272534/?ref_=ad...,2019,tt10272534,-
23,Un divan à Tunis,https://www.imdb.com/title/tt9648886/?ref_=adv...,2019,tt9648886,-
31,Deux,https://www.imdb.com/title/tt9845110/?ref_=adv...,2019,tt9845110,-
41,Alice,https://www.imdb.com/title/tt9120416/?ref_=adv...,2019,tt9120416,-
45,Adoration,https://www.imdb.com/title/tt7715270/?ref_=adv...,2019,tt7715270,-
...,...,...,...,...,...
869,Faux Contact,https://www.imdb.com/title/tt13435054/?ref_=ad...,2021,tt13435054,-
870,Dark West - Director's Cut,https://www.imdb.com/title/tt15731894/?ref_=ad...,2021,tt15731894,-
871,Recuerda - un moment por ella,https://www.imdb.com/title/tt15227746/?ref_=ad...,2021,tt15227746,-
872,Klon: Nouveau Genre,https://www.imdb.com/title/tt14852738/?ref_=ad...,2021,tt14852738,-


#### Pour retrouver les films ayant une année de sortie différentes.

In [54]:
corresp_id3=[]
flat2=[]
for t,a,i in zip(df2['titre'],df2['annee_sortie'],df2['id_imdb']) :
    if a=='2019':
        a='2020'
        url_mdb = 'http://api.themoviedb.org/3/search/movie?api_key={token à ajouter}&query='+t+'&primary_release_year='+a+'&language=fr-FR&page=1&include_adult=false'
        response_l = requests.get(url_mdb)
        json_data_l = response_l.json()
        df_n=pd.json_normalize(json_data_l)
        #print(df_n.columns)
        if len(df_n.columns) == 1 :#not isinstance(df_n, pd.DataFrame)
            flat2.append({t,a,i})
        else:
            for x in df_n.results :
                results=pd.json_normalize(x)
                #print(type(results))
                #print(results.id)
                if x ==[] or isinstance(x, float) :
                    flat2.append({t,a})
                else:
                    corresp_id3.append(results.id)
    elif a=='2020':
        a='2021'
        url_mdb = 'http://api.themoviedb.org/3/search/movie?api_key={token à ajouter}&query='+t+'&primary_release_year='+a+'&language=fr-FR&page=1&include_adult=false'
        response_l = requests.get(url_mdb)
        json_data_l = response_l.json()
        df_n=pd.json_normalize(json_data_l)
        #print(df_n.columns)
        if len(df_n.columns) == 1 :#not isinstance(df_n, pd.DataFrame)
            flat2.append({t,a,i})
        else:
            for x in df_n.results :
                results=pd.json_normalize(x)
                #print(type(results))
                #print(results.id)
                if x ==[] or isinstance(x, float) :
                    flat2.append({t,a})
                else:
                    corresp_id3.append(results.id)
    elif a=='2021':
        a='2020'
        url_mdb = 'http://api.themoviedb.org/3/search/movie?api_key={token à ajouter}&query='+t+'&primary_release_year='+a+'&language=fr-FR&page=1&include_adult=false'
        response_l = requests.get(url_mdb)
        json_data_l = response_l.json()
        df_n=pd.json_normalize(json_data_l)
        #print(df_n.columns)
        if len(df_n.columns) == 1 :#not isinstance(df_n, pd.DataFrame)
            flat2.append({t,a,i})
        else:
            for x in df_n.results :
                results=pd.json_normalize(x)
                #print(type(results))
                #print(results.id)
                if x ==[] or isinstance(x, float) :
                    flat2.append({t,a})
                else:
                    corresp_id3.append(results.id)
    elif a=='2020':
        a='2019'
        url_mdb = 'http://api.themoviedb.org/3/search/movie?api_key={token à ajouter}&query='+t+'&primary_release_year='+a+'&language=fr-FR&page=1&include_adult=false'
        response_l = requests.get(url_mdb)
        json_data_l = response_l.json()
        df_n=pd.json_normalize(json_data_l)
        #print(df_n.columns)
        if len(df_n.columns) == 1 :#not isinstance(df_n, pd.DataFrame)
            flat2.append({t,a,i})
        else:
            for x in df_n.results :
                results=pd.json_normalize(x)
                #print(type(results))
                #print(results.id)
                if x ==[] or isinstance(x, float) :
                    flat2.append({t,a})
                else:
                    corresp_id3.append(results.id)

In [55]:
len(flat2)

242

In [56]:
liste_id_moviedb_temp2=pd.DataFrame(corresp_id3)
liste_id_moviedb_temp2

liste_id_moviedb2=[]
for col in liste_id_moviedb_temp2:
    for y in liste_id_moviedb_temp2[col]:
        if len(str(y))>3:
            liste_id_moviedb2.append(y)

In [57]:
idimbd_moviebd_liste2=[]
ap=('') ## token à ajouter

for id in liste_id_moviedb2:
    url_mdb = 'https://api.themoviedb.org/3/movie/'+str(id)+'/external_ids?api_key='+ap+''
    response_l = requests.get(url_mdb)
    json_data_l = response_l.json()
    #print(json_data_l)
    idimbd_moviebd_liste2.append(json_data_l)

In [58]:
idimbd_moviebd_df3=pd.json_normalize(idimbd_moviebd_liste2)
idimbd_moviebd_df3.head()

,id,imdb_id,facebook_id,instagram_id,twitter_id
0,599377,tt10272534,None,None,None
1,582897,tt9648886,None,None,None
2,621744,tt9845110,deuxlefilm,None,None
3,576002,tt9120416,None,None,None
4,487631,tt7715270,None,None,None


In [59]:
idimbd_moviebd_df4=idimbd_moviebd_df3.copy()
idimbd_moviebd_df4=idimbd_moviebd_df4.drop_duplicates(subset=['id','imdb_id'])
id_moviedb2=[]
aa2={}
for id, idc in zip(idimbd_moviebd_df4.imdb_id,idimbd_moviebd_df4.id):
    aa2[id]=idc

for i in df.id_imdb:
    if i in [x for x in idimbd_moviebd_df4['imdb_id']]:
        id_moviedb2.append(aa2[i])
    else:
        id_moviedb2.append('-')

df['id_moviedb2']=id_moviedb2
df

,titre,url_fiche,annee_sortie,id_imdb,id_moviedb,id_moviedb2
0,La belle époque,https://www.imdb.com/title/tt9172422/?ref_=adv...,2019,tt9172422,595975,-
1,Anna,https://www.imdb.com/title/tt7456310/?ref_=adv...,2019,tt7456310,484641,-
2,Portrait de la jeune fille en feu,https://www.imdb.com/title/tt8613070/?ref_=adv...,2019,tt8613070,531428,-
3,Curiosa,https://www.imdb.com/title/tt7598076/?ref_=adv...,2019,tt7598076,487616,-
4,Le Chant du loup,https://www.imdb.com/title/tt7458762/?ref_=adv...,2019,tt7458762,484468,-
...,...,...,...,...,...,...
870,Dark West - Director's Cut,https://www.imdb.com/title/tt15731894/?ref_=ad...,2021,tt15731894,-,-
871,Recuerda - un moment por ella,https://www.imdb.com/title/tt15227746/?ref_=ad...,2021,tt15227746,-,-
872,Klon: Nouveau Genre,https://www.imdb.com/title/tt14852738/?ref_=ad...,2021,tt14852738,-,-
873,Les Etoiles de Fontevraud,https://www.imdb.com/title/tt18556416/?ref_=ad...,2021,tt18556416,-,-


In [60]:
df2 = df.loc[(df['id_moviedb']=='-') & (df['id_moviedb2']=='-')]
df2

,titre,url_fiche,annee_sortie,id_imdb,id_moviedb,id_moviedb2
59,Avantages de Voyager en Train,https://www.imdb.com/title/tt8406738/?ref_=adv...,2019,tt8406738,-,-
64,#jesuislà,https://www.imdb.com/title/tt9828918/?ref_=adv...,2019,tt9828918,-,-
153,Entre deux trains,https://www.imdb.com/title/tt9133200/?ref_=adv...,2019,tt9133200,-,-
173,Flying Guy,https://www.imdb.com/title/tt8253030/?ref_=adv...,2019,tt8253030,-,-
197,Fragments,https://www.imdb.com/title/tt9913418/?ref_=adv...,2019,tt9913418,-,-
...,...,...,...,...,...,...
869,Faux Contact,https://www.imdb.com/title/tt13435054/?ref_=ad...,2021,tt13435054,-,-
870,Dark West - Director's Cut,https://www.imdb.com/title/tt15731894/?ref_=ad...,2021,tt15731894,-,-
871,Recuerda - un moment por ella,https://www.imdb.com/title/tt15227746/?ref_=ad...,2021,tt15227746,-,-
872,Klon: Nouveau Genre,https://www.imdb.com/title/tt14852738/?ref_=ad...,2021,tt14852738,-,-


In [61]:
vide=[]
for a,b in zip (df.id_moviedb,df.id_moviedb2):
    if a=='-' and b=='-':
        vide.append('-')
    elif a!='-' and b=='-':
        vide.append(a)
    elif a=='-' and b!='-':
        vide.append(b)
    else :
        vide.append('-')
df['id_moviedb_final']=vide

df3 = df.loc[df['id_moviedb_final']=='-']
len(df3)

280

In [62]:
df_final = df.copy().drop(columns=['id_moviedb', 'id_moviedb2'])

#### Je concatene les 2 tables de correspondance id

In [63]:
frames = [idimbd_moviebd_df3, idimbd_moviebd_df]
table_externals_id = pd.concat(frames).drop_duplicates()
table_externals_id

,id,imdb_id,facebook_id,instagram_id,twitter_id
0,599377,tt10272534,None,None,None
1,582897,tt9648886,None,None,None
2,621744,tt9845110,deuxlefilm,None,None
3,576002,tt9120416,None,None,None
4,487631,tt7715270,None,None,None
...,...,...,...,...,...
1506,799078,None,None,None,None
1507,844135,tt5806814,None,None,None
1508,751088,None,None,None,None
1509,716875,None,None,None,None


Je lance la recherche pour avoir la liste du casting et celle de l'équipe technique à l'aide de l'api adéquate

In [64]:
film_credit=[]
for f in df_final.id_moviedb_final:
    url_mdb = 'https://api.themoviedb.org/3/movie/'+str(f)+'/credits?api_key={token à ajouter}&language=fr-FR'
    response_l = requests.get(url_mdb)
    json_data_l = response_l.json()
    film_credit.append(json_data_l)

In [65]:
film_credit_df=pd.json_normalize(film_credit)
film_credit_df.head()

,id,cast,crew,success,status_code,status_message
0,595975.0,"[{'adult': False, 'gender': 2, 'id': 6012, 'kn...","[{'adult': False, 'gender': 2, 'id': 4284, 'kn...",NaN,NaN,NaN
1,484641.0,"[{'adult': False, 'gender': 1, 'id': 1411561, ...","[{'adult': False, 'gender': 2, 'id': 59, 'know...",NaN,NaN,NaN
2,531428.0,"[{'adult': False, 'gender': 1, 'id': 1385600, ...","[{'adult': False, 'gender': 0, 'id': 14604, 'k...",NaN,NaN,NaN
3,487616.0,"[{'adult': False, 'gender': 1, 'id': 1385600, ...","[{'adult': False, 'gender': 2, 'id': 4401, 'kn...",NaN,NaN,NaN
4,484468.0,"[{'adult': False, 'gender': 2, 'id': 78124, 'k...","[{'adult': False, 'gender': 2, 'id': 2424, 'kn...",NaN,NaN,NaN


In [67]:
cast = film_credit_df[['id','cast']].loc[film_credit_df.status_code!=34]
encore_list=[]
encore_id=[]
for id, cast in zip(cast.id,cast.cast):
    #print(id)
    for ct in cast:
        #d=pd.json_normalize(c)
        encore_list.append(pd.json_normalize(ct))
        encore_id.append(id)
    
final_cast_list=pd.concat(encore_list)      
final_cast_list['id_film']=encore_id

In [68]:
final_cast_list

,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,cast_id,character,credit_id,order,id_film
0,False,2,6012,Acting,Daniel Auteuil,Daniel Auteuil,8.529,/5asrvkdSnlYlHap3ldDNyE5opeI.jpg,2,Victor Drumond,5cc71ecc92514119e5f5b47b,0,595975.0
0,False,2,19866,Acting,Guillaume Canet,Guillaume Canet,12.378,/qy0xe5k8tz2K7HYArBIfyKN90Vj.jpg,3,Antoine,5cc71ed5c3a368493683de08,1,595975.0
0,False,1,1280476,Acting,Doria Tillier,Doria Tillier,8.182,/kCS73s2jtUY7dV7eDVKOAZjvgVS.jpg,4,Margot,5cc71edec3a368467783b4d7,2,595975.0
0,False,1,20234,Acting,Fanny Ardant,Fanny Ardant,17.104,/q8Hj88E3ME5z795haGMqVDwFDjp.jpg,5,Marianne Drumond,5cc71ee7c3a368282083c4aa,3,595975.0
0,False,2,24816,Acting,Pierre Arditi,Pierre Arditi,5.046,/kLvyrJkfcGUKzn3rHI0SYX5uPwj.jpg,6,Pierre,5cc71ef00e0a2678a0f02c36,4,595975.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,False,0,3464083,Acting,Alexis Desprez,Alexis Desprez,0.600,None,6,,622e0a850e441900451f6689,2,949097.0
0,False,0,3464084,Acting,Anass Radouani,Anass Radouani,0.600,None,7,,622e0a9ce04aca0078ac1a08,3,949097.0
0,False,0,3464085,Acting,Baba Diallo,Baba Diallo,0.600,None,8,,622e0aa12a210c00784b3486,4,949097.0
0,False,0,3464086,Acting,Franck Fondeur,Franck Fondeur,0.600,None,9,,622e0aa7e04aca0078ac1a86,5,949097.0


In [69]:
crew = film_credit_df[['id','crew']].loc[film_credit_df.status_code!=34]
crew_list=[]
crew_id=[]
for id, crew in zip(crew.id,crew.crew):
    #print(id)
    for cw in crew:
        #d=pd.json_normalize(c)
        crew_list.append(pd.json_normalize(cw))
        crew_id.append(id)
    
final_crew_list=pd.concat(crew_list)      
final_crew_list['id_film']=crew_id
final_crew_list

,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,credit_id,department,job,id_film
0,False,2,4284,Art,Stéphane Rozenbaum,Stéphane Rozenbaum,0.600,/pfst2G1aIWZQ9vfyAJjtYD3bpPD.jpg,619ebd04f90b19004286e61f,Art,Production Design,595975.0
0,False,2,47818,Sound,Jean-Paul Hurier,Jean-Paul Hurier,5.041,/euXWeZr65R15bKL8OKQGag06HVW.jpg,61a0d1c2ed28b90066261de8,Sound,Sound Re-Recording Mixer,595975.0
0,False,1,56407,Editing,Anny Danché,Anny Danché,1.400,None,5cc71f549251412ba1fdbdf4,Editing,Editor,595975.0
0,False,2,543839,Production,François Kraus,François Kraus,0.600,/lzmXabwB0hwopxLVMPbTaCmmVb.jpg,5cc7204b0e0a264eefee46db,Production,Producer,595975.0
0,False,2,543840,Production,Denis Pineau-Valencienne,Denis Pineau-Valencienne,0.600,/kWvyVPbHgHBa9YXh8cATSiKNhwr.jpg,5cc720590e0a267f78ef0d88,Production,Producer,595975.0
...,...,...,...,...,...,...,...,...,...,...,...,...
0,False,0,2272935,Directing,Jacques Meilleurat,Jacques Meilleurat,0.600,None,60f9a8e79603310081181506,Crew,Cinematography,853152.0
0,False,0,2554116,Production,Jean-Christophe Meneec,Jean-Christophe Meneec,0.600,None,622e0a6cb04605007a7bc6d2,Writing,Screenplay,949097.0
0,False,0,2554116,Production,Jean-Christophe Meneec,Jean-Christophe Meneec,0.600,None,622e0a662866fa00454f6e67,Directing,Director,949097.0
0,False,0,2554116,Production,Jean-Christophe Meneec,Jean-Christophe Meneec,0.600,None,622e0a724e4dff00791a7451,Production,Producer,949097.0


In [70]:
len(final_cast_list),len(final_crew_list)

(7943, 7384)

### Recherche des budgets de films 
je passe par les infos dans Movie DB mais il faurait faire un vérification à l'aide des doc officiels du CNC

In [79]:
film_details=[]
for f in df_final.id_moviedb_final:
    url_mdb ='https://api.themoviedb.org/3/movie/'+str(f)+'?api_key='+ap+'&language=fr-FR'
    response_l = requests.get(url_mdb)
    json_data_l = response_l.json()
    film_details.append(json_data_l)

In [82]:
film_details_df=pd.json_normalize(film_details)
#film_details_df.head()

In [84]:
film_details_df.columns

Index(['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres',
       'homepage', 'id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'belongs_to_collection.id',
       'belongs_to_collection.name', 'belongs_to_collection.poster_path',
       'belongs_to_collection.backdrop_path', 'success', 'status_code',
       'status_message'],
      dtype='object')

In [89]:
detail = film_details_df.loc[film_details_df.status_code!=34]
dico_budget={}
budget_list=[]

#l'idée est ici d'intégrer le budget dans la dataframe final "df_final"
#je crée dans un premier temps un dico key=id du film, value=le budget du film
for id, budget in zip(detail.id,detail.budget):
    dico_budget[id]=int(budget)

#boucle sur les id_moviedb du dataframe final pour trouver le budget correspondant dans le df "detail" 
    #puis l'intégrer dans la liste budget_list
for i in df_final.id_moviedb_final:
    if i in [x for x in detail['id']]:
        budget_list.append(dico_budget[i])
    else:
        budget_list.append('-')

#je crée la colonne "budget" dans le dataframe final à l'aide de la liste budget_list
df_final['budget']=budget_list

In [90]:
df_final.head(3)

,titre,url_fiche,annee_sortie,id_imdb,id_moviedb_final,budget
0,La belle époque,https://www.imdb.com/title/tt9172422/?ref_=adv...,2019,tt9172422,595975,11000000
1,Anna,https://www.imdb.com/title/tt7456310/?ref_=adv...,2019,tt7456310,484641,34000000
2,Portrait de la jeune fille en feu,https://www.imdb.com/title/tt8613070/?ref_=adv...,2019,tt8613070,531428,5740000


In [91]:
writer = pd.ExcelWriter('movies_'+str(dt.datetime.today().strftime("%Y%m%d"))+'.xlsx', engine='xlsxwriter')

df_final.to_excel(writer, sheet_name='liste_imdb_2019-2021', index=False)
table_externals_id.to_excel(writer, sheet_name='correspondance_id', index=False)
final_cast_list.to_excel(writer, sheet_name='liste_casting', index=False)
final_crew_list.to_excel(writer, sheet_name='liste_equipe_technique', index=False)
writer.save()

In [92]:
df_final.to_csv('liste_imdb_2019-2021.csv', index=False)
table_externals_id.to_csv('correspondance_id.csv', index=False)
final_cast_list.to_csv('liste_casting.csv', index=False)
final_crew_list.to_csv('liste_equipe_technique.csv', index=False)